<a href="https://colab.research.google.com/github/torimonn/disclosure/blob/main/main_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 財務諸表抽出ノートブック
このノートブックでは、Google Generative AI (Gemini) を用いてPDFから
貸借対照表と損益計算書を抽出し、CSVとして保存する処理を解説付きで実行します。

## 必要なライブラリのインストールとインポート
Colab 環境では下記のようにライブラリをインストールしてから実行してください。

In [1]:
!pip install -qU google-genai pydantic pypdf pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 852.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.0/200.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 23.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.0 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.


In [2]:
import os
import json
import time
from datetime import date
from typing import List, Optional, Union

from google.colab import userdata
import google.colab.files  # PDF アップロード用
import google.generativeai as genai
from pydantic import BaseModel, Field, ValidationError
from pypdf import PdfReader
import pandas as pd

try:
    google_api_key = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=google_api_key)
    print('APIキーの設定が完了しました。')
except userdata.SecretNotFoundError as e:
    print(f"エラー: Colab Secrets に 'GOOGLE_API_KEY' が見つかりません: {e}")
    print("Google Colab の Secrets に 'GOOGLE_API_KEY' を設定してください。")
    raise SystemExit
except Exception as e:
    print(f'APIキー設定中に予期せぬエラーが発生しました: {e}')
    raise SystemExit


APIキーの設定が完了しました。


### PDFファイルのアップロード
以下のセルを実行するとファイル選択ダイアログが表示されます。処理したいPDFを1枚以上アップロードしてください。

In [3]:
uploaded = google.colab.files.upload()
pdf_paths = [os.path.join('/content', name) for name in uploaded.keys()]
print('アップロードされたPDF:', pdf_paths)

Saving disclosure_2024.pdf to disclosure_2024.pdf
Saving disclo2024_0.pdf to disclo2024_0.pdf
Saving all.pdf to all.pdf
Saving tamashin_disclo2024.pdf to tamashin_disclo2024.pdf
Saving disclo24.pdf to disclo24.pdf
アップロードされたPDF: ['/content/disclosure_2024.pdf', '/content/disclo2024_0.pdf', '/content/all.pdf', '/content/tamashin_disclo2024.pdf', '/content/disclo24.pdf']


In [4]:
MODEL_NAME = "gemini-2.5-flash-preview-05-20"

In [5]:
class PDFMetadata(BaseModel):
    """PDFから抽出するメタデータ。貸借対照表と損益計算書の金額単位を別々に保持"""
    company_name_japanese: str = Field(description="会社名（日本語）")
    company_name_english: Optional[str] = Field(default=None, description="会社名（英語）")
    balance_sheet_pages_1_indexed: List[int] = Field(description="貸借対照表ページ番号リスト")
    income_statement_pages_1_indexed: Optional[List[int]] = Field(default=None, description="損益計算書ページ番号リスト")
    estimated_balance_sheet_type: Optional[str] = Field(default=None, description="貸借対照表の種類推定")
    estimated_income_statement_type: Optional[str] = Field(default=None, description="損益計算書の種類推定")
    balance_sheet_amount_unit: int = Field(description="貸借対照表の金額単位。百万円なら1000000等")
    income_statement_amount_unit: Optional[int] = Field(default=None, description="損益計算書の金額単位。百万円なら1000000等")


In [6]:
class StatementItem(BaseModel):
    name_japanese: str
    name_english: str
    value: Optional[Union[int, float]]
    indent_level: int
    children: List['StatementItem'] = Field(default_factory=list)

In [7]:
# Pydantic v2 では update_forward_refs() の代わりに model_rebuild() を使用
StatementItem.model_rebuild()

In [8]:
class Section(BaseModel):
    total_value: Optional[Union[int, float]] = None
    items: List[StatementItem]

In [9]:
class FiscalYearBalanceSheet(BaseModel):
    end_date: date
    description: Optional[str] = None
    assets: Section
    liabilities: Section
    net_assets: Section

In [10]:
class BalanceSheetBody(BaseModel):
    fiscal_year_data: List[FiscalYearBalanceSheet]

In [11]:
class BalanceSheetResponse(BaseModel):
    balance_sheet: BalanceSheetBody

In [12]:
class FiscalYearIncomeStatement(BaseModel):
    end_date: date
    description: Optional[str] = None
    items: List[StatementItem]

In [13]:
class IncomeStatementBody(BaseModel):
    fiscal_year_data: List[FiscalYearIncomeStatement]

In [14]:
class IncomeStatementResponse(BaseModel):
    income_statement: IncomeStatementBody

In [15]:
def upload_pdf_to_genai(pdf_file_path: str) -> Optional[genai.protos.FileData]:
    try:
        return genai.upload_file(pdf_file_path)
    except Exception as e:
        print(f"PDFのアップロードに失敗しました: {e}")
        return None

In [16]:
def extract_text_from_pdf_pypdf(pdf_file_path: str, pages_0_indexed: Optional[List[int]]) -> str:
    try:
        reader = PdfReader(pdf_file_path)
        if pages_0_indexed is None:
            pages = range(len(reader.pages))
        else:
            pages = pages_0_indexed
        texts = [reader.pages[p].extract_text() or "" for p in pages]
        return "\n".join(texts)
    except Exception as e:
        print(f"PDFテキスト抽出に失敗しました: {e}")
        return ""

In [18]:
def create_generative_content_parts_for_metadata(uploaded_pdf_file: genai.protos.FileData, pydantic_schema: dict) -> List[Union[str, genai.protos.FileData]]:
    schema_json = json.dumps(pydantic_schema, ensure_ascii=False, indent=2)
    instruction = ("PDF から会社名、貸借対照表ページ、損益計算書ページ、表の種類、貸借対照表用の金額単位、損益計算書用の金額単位を抽出してください。出力はJSONのみで、スキーマは次の通りです:\n" + schema_json)
    return [instruction, uploaded_pdf_file]

In [19]:
def create_generative_content_parts_for_balance_sheet(uploaded_pdf_file: genai.protos.FileData, company_name_japanese: str, company_name_english: str, pydantic_schema: dict, auxiliary_pdf_text: Optional[str] = None) -> List[Union[str, genai.protos.FileData]]:
    schema_json = json.dumps(pydantic_schema, ensure_ascii=False, indent=2)
    instruction = f"""会社名「{company_name_japanese} ({company_name_english})」の単体貸借対照表を抽出し、次のJSONスキーマに従ってください:\n{schema_json}"""
    parts = [instruction, uploaded_pdf_file]
    if auxiliary_pdf_text:
        parts.append("\n\n### 補助テキスト:\n" + auxiliary_pdf_text)
    return parts

In [20]:
def create_generative_content_parts_for_income_statement(uploaded_pdf_file: genai.protos.FileData, company_name_japanese: str, company_name_english: str, pydantic_schema: dict, auxiliary_pdf_text: Optional[str] = None) -> List[Union[str, genai.protos.FileData]]:
    schema_json = json.dumps(pydantic_schema, ensure_ascii=False, indent=2)
    instruction = f"""会社名「{company_name_japanese} ({company_name_english})」の単体損益計算書を抽出し、次のJSONスキーマに従ってください:\n{schema_json}"""
    parts = [instruction, uploaded_pdf_file]
    if auxiliary_pdf_text:
        parts.append("\n\n### 補助テキスト:\n" + auxiliary_pdf_text)
    return parts

In [21]:
def call_llm_for_structured_output(content_parts: List[Union[str, genai.protos.FileData]], output_model: BaseModel) -> Optional[BaseModel]:
    try:
        model = genai.GenerativeModel(model_name=MODEL_NAME)
        response = model.generate_content(
            contents=content_parts,
            generation_config=genai.GenerationConfig(
                response_mime_type="application/json",
                temperature=0
            ),
            safety_settings={
                'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE',
                'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE',
                'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE',
                'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE',
            }
        )
        text = response.text.strip()
        if text.startswith("```json") and text.endswith("```"):
            text = text[7:-3].strip()
        data = json.loads(text)
        return output_model.model_validate(data)
    except (ValidationError, json.JSONDecodeError) as e:
        print("LLM応答の解析に失敗しました", e)
        print(text)
        return None
    except Exception as e:
        print("LLM呼び出しに失敗しました", e)
        return None

In [22]:
def multiply_unit(items: List[StatementItem], amount_unit: int):
    for it in items:
        if it.value is not None:
            it.value *= amount_unit
        if it.children:
            multiply_unit(it.children, amount_unit)

In [23]:
def build_financial_rows(
    balance_sheet_resp: BalanceSheetResponse,
    income_statement_resp: IncomeStatementResponse,
    company_name: str,
    bs_unit: int,
    pl_unit: int,
    bs_pages: Optional[List[int]],
    pl_pages: Optional[List[int]],
) -> List[tuple]:
    """貸借対照表・損益計算書をCSV用の行リストに変換する"""
    latest_bs = balance_sheet_resp.balance_sheet.fiscal_year_data[0]
    latest_pl = income_statement_resp.income_statement.fiscal_year_data[0]

    multiply_unit(latest_bs.assets.items, bs_unit)
    multiply_unit(latest_bs.liabilities.items, bs_unit)
    multiply_unit(latest_bs.net_assets.items, bs_unit)
    multiply_unit(latest_pl.items, pl_unit)

    bs_pages_str = ',' .join(str(p) for p in bs_pages) if bs_pages else ''
    pl_pages_str = ',' .join(str(p) for p in pl_pages) if pl_pages else ''

    rows: List[tuple] = []
    rows.append(('金庫名', company_name))
    rows.append(('決算期', latest_bs.end_date.strftime('%Y-%m-%d')))
    rows.append(('貸借対照表記載ページ', bs_pages_str))
    rows.append(('損益計算書記載ページ', pl_pages_str))

    def flatten(items, dst):
        for it in items:
            dst.append((it.name_japanese, it.value if it.value is not None else ''))
            if it.children:
                flatten(it.children, dst)

    rows.append(('-- 貸借対照表 --', ''))
    if latest_bs.assets.total_value is not None:
        rows.append(('資産合計', latest_bs.assets.total_value))
    flatten(latest_bs.assets.items, rows)
    if latest_bs.liabilities.total_value is not None:
        rows.append(('負債合計', latest_bs.liabilities.total_value))
    flatten(latest_bs.liabilities.items, rows)
    if latest_bs.net_assets.total_value is not None:
        rows.append(('純資産合計', latest_bs.net_assets.total_value))
    flatten(latest_bs.net_assets.items, rows)

    rows.append(('', ''))
    rows.append(('-- 損益計算書 --', ''))
    flatten(latest_pl.items, rows)
    return rows


In [24]:
def process_pdf(pdf_path: str) -> Optional[List[tuple]]:
    """PDF 1ファイルを処理し、CSV 行データのリストを返す"""
    print(f'処理開始: {pdf_path}')
    uploaded = upload_pdf_to_genai(pdf_path)
    if not uploaded:
        return None
    schema_meta = PDFMetadata.model_json_schema()
    meta_parts = create_generative_content_parts_for_metadata(uploaded, schema_meta)
    metadata: Optional[PDFMetadata] = call_llm_for_structured_output(meta_parts, PDFMetadata)
    try:
        genai.delete_file(uploaded.name)
    except Exception:
        pass
    if not metadata:
        print('メタデータの取得に失敗しました')
        return None

    company_jp = metadata.company_name_japanese
    company_en = metadata.company_name_english or company_jp
    bs_unit = metadata.balance_sheet_amount_unit
    pl_unit = metadata.income_statement_amount_unit or bs_unit

    pages_bs_list = metadata.balance_sheet_pages_1_indexed or []
    pages_pl_list = metadata.income_statement_pages_1_indexed or []

    pages_bs = [p - 1 for p in pages_bs_list] if pages_bs_list else None
    pages_pl = [p - 1 for p in pages_pl_list] if pages_pl_list else None

    uploaded_bs = upload_pdf_to_genai(pdf_path)
    aux_text_bs = extract_text_from_pdf_pypdf(pdf_path, pages_bs)
    schema_bs = BalanceSheetResponse.model_json_schema()
    parts_bs = create_generative_content_parts_for_balance_sheet(
        uploaded_bs, company_jp, company_en, schema_bs, aux_text_bs
    )
    bs_resp: Optional[BalanceSheetResponse] = call_llm_for_structured_output(parts_bs, BalanceSheetResponse)
    try:
        genai.delete_file(uploaded_bs.name)
    except Exception:
        pass
    if not bs_resp:
        print('貸借対照表の取得に失敗しました')
        return None

    uploaded_pl = upload_pdf_to_genai(pdf_path)
    aux_text_pl = extract_text_from_pdf_pypdf(pdf_path, pages_pl)
    schema_pl = IncomeStatementResponse.model_json_schema()
    parts_pl = create_generative_content_parts_for_income_statement(
        uploaded_pl, company_jp, company_en, schema_pl, aux_text_pl
    )
    pl_resp: Optional[IncomeStatementResponse] = call_llm_for_structured_output(parts_pl, IncomeStatementResponse)
    try:
        genai.delete_file(uploaded_pl.name)
    except Exception:
        pass
    if not pl_resp:
        print('損益計算書の取得に失敗しました')
        return None

    rows = build_financial_rows(
        bs_resp,
        pl_resp,
        company_jp,
        bs_unit,
        pl_unit,
        pages_bs_list,
        pages_pl_list,
    )
    return rows


In [25]:
def main():
    api_key = os.environ.get('GOOGLE_API_KEY')
    if not api_key:
        print('環境変数 GOOGLE_API_KEY を設定してください')
        return
    genai.configure(api_key=api_key)

    if len(sys.argv) < 2:
        print('PDFファイルを指定してください')
        return
    all_rows: List[tuple] = []
    for pdf_path in sys.argv[1:]:
        rows = process_pdf(pdf_path)
        if rows:
            if all_rows:
                all_rows.append(('', ''))
            all_rows.extend(rows)

    if not all_rows:
        print('処理結果がありません')
        return

    df = pd.DataFrame(all_rows, columns=['科目', '金額(円)'])
    output_path = 'financials.csv'
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f'CSVを保存しました: {output_path}')


In [26]:
all_rows = []
for pdf in pdf_paths:
    rows = process_pdf(pdf)
    if rows:
        if all_rows:
            all_rows.append(('', ''))
        all_rows.extend(rows)

if all_rows:
    df = pd.DataFrame(all_rows, columns=['科目', '金額(円)'])
    df.to_csv('financials.csv', index=False, encoding='utf-8-sig')
    print('CSVを保存しました: financials.csv')
else:
    print('処理結果がありません')


処理開始: /content/disclosure_2024.pdf
CSVを保存しました: disclosure_2024.csv
処理開始: /content/disclo2024_0.pdf
CSVを保存しました: disclo2024_0.csv
処理開始: /content/all.pdf
CSVを保存しました: all.csv
処理開始: /content/tamashin_disclo2024.pdf
CSVを保存しました: tamashin_disclo2024.csv
処理開始: /content/disclo24.pdf
CSVを保存しました: disclo24.csv


### 複数のCSVを横に連結したい

`merge_financials.py` は各信金向けに生成した `financials.csv` を、勘定科目を基準に外部結合して横並びに整形するスクリプトです。
先頭4行のメタ情報を除いた 5 行目以降の `科目` と `金額(円)` を利用して結合します。

```bash
python merge_financials.py 信金A.csv 信金B.csv 信金C.csv
```

実行すると `merged_financials.csv` に1列目:勘定科目、2列目以降:金庫名を列見出しとした金額が並びます。
科目が存在しない場合は自動的に空欄になります。


In [ ]:
!python merge_financials.py 信金A.csv 信金B.csv 信金C.csv